Let's get the stats that maybe everyone can agree on as a start


In [1]:
import numpy as np
import matplotlib.pylab as plt
%matplotlib inline
from rubin_sim.satellite_constellations import (Constellation, starlink_tles_v2,
                                                oneweb_tles,
                                                ModelObservatory,
                                                SatelliteAvoidBasisFunction)
import healpy as hp
from rubin_sim.data import get_baseline
import pandas as pd
import sqlite3
import os

In [2]:
save_file = 'streaks_all.npz'

In [3]:
# a starlink gen 2 + oneweb
tles = starlink_tles_v2() #+ oneweb_tles()
const = Constellation(tles)
print('number of satellites= ', len(tles))

number of satellites=  29988


In [4]:
baseline = get_baseline()
con = sqlite3.connect(baseline)
# year 1
visits = pd.read_sql('select fieldRA, fieldDec, fivesigmadepth, observationstartmjd,rotSkyPos,visitExposureTime from observations where night < 365', con)
con.close()

In [5]:
visits

,fieldRA,fieldDec,fiveSigmaDepth,observationStartMJD,rotSkyPos,visitExposureTime
0,298.603526,-15.574752,22.546480,60218.001806,224.242758,30.0
1,297.035243,-13.144502,22.532127,60218.002260,231.080063,30.0
2,295.418862,-15.854174,22.546281,60218.002715,238.282952,30.0
3,356.288005,-51.164810,23.128326,60218.004914,129.375130,30.0
4,0.158759,-53.139864,23.075882,60218.005362,130.108353,30.0
...,...,...,...,...,...,...
216115,127.418325,-9.886886,22.575541,60582.395343,147.208171,15.0
216116,125.666648,-7.743953,22.589142,60582.395579,149.420262,15.0
216117,127.022962,-4.908917,22.458614,60582.395820,150.936373,15.0
216118,130.532609,-9.181433,22.447880,60582.396063,146.931615,15.0


In [6]:
keys = ['fieldRA', 'fieldDec', 'fiveSigmaDepth', 'observationStartMJD', 'rotSkyPos', 'visitExposureTime']
types = [float] * len(keys)

numpy_visits = np.zeros(np.size(visits["fieldRA"].values), dtype=list(zip(keys,types)))

for key in keys:
    numpy_visits[key] = visits[key].values



In [7]:
if os.path.isfile(save_file):
    data = np.load(save_file)
    streak_lengths = data['streak_lengths'].copy()
    n_streaks = data['n_streaks'].copy()
    data.close()
#else:
    #streak_lengths, n_streaks = const.check_pointings(visits["fieldRA"].values,
    #                                                     visits["fieldDec"].values,
    #                                                     visits["observationStartMJD"].values, visit_time=32.)
    #np.savez(save_file, streak_lengths=streak_lengths, n_streaks=n_streaks)

In [8]:
n_streaks

array([3, 0, 0, ..., 0, 0, 0])

In [9]:
np.shape(n_streaks)

(215980,)

In [10]:
# save so we don't have to grind that out again
#if not os.path.isfile('streaks.npz'):
#    np.savez('streaks.npz', streak_lengths=streak_lengths, n_streaks=n_streaks)

In [11]:
# not quite right because I have 
exptimes = numpy_visits['visitExposureTime']+0
exptime_before = np.sum(exptimes)
exptimes[np.where(n_streaks > 0)] = exptimes[np.where(n_streaks > 0)]/2.
exptime_after = np.sum(exptimes)

print('Fractional loss of total exposure=', (exptime_before-exptime_after)/exptime_before)


Fractional loss of total exposure= 0.03839980880929899


In [12]:
# what fraction of pixels are lost to streaks with a 1 arcmin mask
# 
pixscale = 0.2 # arcsec/pix
tot_pix = 3.2e9 # total pixels in focal plane
mask_width = 60 # arcsec

print('fraction of pix lost', np.sum(streak_lengths/pixscale*3600 * mask_width /pixscale)/(tot_pix*np.size(streak_lengths)))

fraction of pix lost 0.00041554225423248443


In [13]:
# fraction of images with a streak
print('fraction of streaked images= ', np.where(n_streaks > 0)[0].size/np.size(n_streaks))

fraction of streaked images=  0.07875729234188351


In [14]:
print('mean streak length', np.mean(streak_lengths)*60, ' (arcmin)') # to arcmin

mean streak length 14.774835706043891  (arcmin)


In [15]:
non_neo = np.where(numpy_visits["visitExposureTime"] > 20)[0]

In [ ]:
print('fraction of pix lost non-NEO', np.sum(streak_lengths[non_neo]/pixscale*3600 * mask_width /pixscale)/(tot_pix*np.size(streak_lengths[non_neo])))